In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hobart,AU,2020-08-31 23:02:09,-42.88,147.33,44.01,64,75,10.29
1,1,Victoria,HK,2020-08-31 23:03:09,22.29,114.16,84.00,79,4,5.01
2,2,Pacifica,US,2020-08-31 23:05:16,37.61,-122.49,73.99,60,40,13.87
3,3,Mataura,NZ,2020-08-31 23:05:16,-46.19,168.86,45.00,72,88,11.01
4,4,Touros,BR,2020-08-31 23:05:16,-5.20,-35.46,77.00,78,40,9.17


In [5]:
#Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Victoria,HK,2020-08-31 23:03:09,22.29,114.16,84.00,79,4,5.01
4,4,Touros,BR,2020-08-31 23:05:16,-5.20,-35.46,77.00,78,40,9.17
6,6,Virginia Beach,US,2020-08-31 23:05:17,36.85,-75.98,80.01,70,76,1.01
12,12,Georgetown,MY,2020-08-31 23:00:36,5.41,100.34,78.80,94,20,5.39
13,13,San Patricio,MX,2020-08-31 23:05:18,19.22,-104.70,86.00,74,75,11.41
15,15,Vaini,TO,2020-08-31 23:05:19,-21.20,-175.20,80.60,83,20,16.11
20,20,Amahai,ID,2020-08-31 23:05:20,-3.33,128.92,84.99,88,94,1.01
23,23,Butaritari,KI,2020-08-31 23:05:20,3.07,172.79,84.06,72,100,8.23
27,27,Atuona,PF,2020-08-31 23:05:21,-9.80,-139.03,79.00,76,59,23.62
29,29,Lorengau,PG,2020-08-31 23:05:22,-2.02,147.27,83.61,74,22,3.87


In [12]:
preferred_cities_df.count()

#preferred_cities_df.counts()

#preferred_cities_df.isnull().sum()

#preferred_cities_df.notnull().sum()

City_ID       190
City          190
Country       190
Date          190
Lat           190
Lng           190
Max Temp      190
Humidity      190
Cloudiness    190
Wind Speed    190
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Victoria,HK,84.00,22.29,114.16,
4,Touros,BR,77.00,-5.20,-35.46,
6,Virginia Beach,US,80.01,36.85,-75.98,
12,Georgetown,MY,78.80,5.41,100.34,
13,San Patricio,MX,86.00,19.22,-104.70,
15,Vaini,TO,80.60,-21.20,-175.20,
20,Amahai,ID,84.99,-3.33,128.92,
23,Butaritari,KI,84.06,3.07,172.79,
27,Atuona,PF,79.00,-9.80,-139.03,
29,Lorengau,PG,83.61,-2.02,147.27,


In [14]:
# Import the API key.
from config import g_key

# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

NameError: name 'hotel_info' is not defined